In [1]:
import requests
import pandas as pd

In [2]:
API_URL= 'http://stats.oecd.org/SDMX-JSON/data/'

In [3]:
params = {
    'mandatory': {'dataSet': 'PAT_DEV', 'filter': 'all', 'agency': 'all'},
    'optional': {},
}

In [4]:
def get_raw_data(API_URL, params):
    mandatory = params['mandatory']
    optional = params['optional']
    url = f"{API_URL}/{mandatory['dataSet']}/{mandatory['filter']}/{mandatory['agency']}"
    
    response = requests.get(url, params=optional)
    
    return response.json(), response.url

In [6]:
def parse_key(key):
    split = key.split(':')
    return {f'key_{i}': int(k) for i,k in enumerate(split)}


def parse_series(series):
    
    dfs = []
    
    for serie_key, serie in series.items():
        key_idx = parse_key(serie_key)
        
        df = pd.DataFrame.from_dict(serie['observations'], orient='index').rename(columns = {0: 'Value'})
        df.index = df.index.astype(int)
        
        for key, idx in key_idx.items():
            df[key] = idx
        
        dfs.append(df)  
    
    return pd.concat(dfs)


def merge_dims_series(series_df, structure_dims):
    '''To improve, special case for TIME_PERIOD seems risky'''
    df = series_df.merge(structure_dims['TIME_PERIOD'], left_index=True, right_index=True)
    
    for dim, dim_df in structure_dims.items():
        if dim != 'TIME_PERIOD':
            df = df.merge(dim_df, on=dim)
    return df  



def parse_structure_dims(structure_dims):
    '''To improve, check what exactly observation look like across datasets'''
    dfs = {}
    
    for dim in structure_dims['series']:
        
        key = f"key_{dim['keyPosition']}"
        df = pd.DataFrame(dim['values']).rename(columns={'id': dim['id'], 'name': dim['name']}).reset_index().rename(columns={'index':key})
        
        dfs[key] = df
    
    for obs in structure_dims['observation']:
        
        dfs[obs['role']] = pd.DataFrame(obs['values']).rename(columns={'id': obs['id'], 'name': obs['name']})

    return dfs

def json_to_pandas(data_json):
    
    structure_dims = data_json['structure']['dimensions']
    series = data_json['dataSets'][0]['series']

    dims = parse_structure_dims(structure_dims)
    series = parse_series(series)
    
    df = merge_dims_series(series, dims)
    
    return df

In [8]:
data, url = get_raw_data(API_URL, params)

In [9]:
df = json_to_pandas(data)

In [10]:
def fomart_pandas(df):
    to_drop = [1,'TIME_PERIOD'] + [col for col in df.columns if 'key_' in str(col)]
    
    df = (
        df.drop(columns=to_drop)
          .rename(columns={'COU': 'ISO'})
    )
    
    return df

In [12]:
test = fomart_pandas(df)

In [17]:
test.query('ISO == "TUR" and DOM == "MAN_AIR_MO" and Year == "2005"')

,Value,Year,ISO,Inventor country,SIZE,Family size,DOM,Technology domain
313132,5.53,2005,TUR,Turkey,ONE,One and greater (all inventions),MAN_AIR_MO,Emissions abatement from mobile sources (e.g. ...
315760,2.03,2005,TUR,Turkey,TWO,Two and greater,MAN_AIR_MO,Emissions abatement from mobile sources (e.g. ...
318071,0.70,2005,TUR,Turkey,THREE,Three and greater,MAN_AIR_MO,Emissions abatement from mobile sources (e.g. ...
320226,0.70,2005,TUR,Turkey,FOUR,Four and greater,MAN_AIR_MO,Emissions abatement from mobile sources (e.g. ...


In [ ]:
test